<a href="https://colab.research.google.com/github/SushmaMacha/SkinLesions/blob/master/CNN_Scratch_hist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s gdrive/'My Drive'/CMPE257/Data data
!ln -s gdrive/'My Drive'/CMPE257/Code code

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


from skimage import data, img_as_float
from skimage import exposure

In [0]:
img_width, img_height = 256, 256

train_data_dir = 'data/train'
validation_data_dir = 'data/val'
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [0]:
from keras.optimizers import SGD

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

sgd = SGD(lr=0.05, momentum=0.9, nesterov=True)

#train_acc around 50% max val_acc 65% :( 20 epochs
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

#model.compile(loss='binary_crossentropy',
#              optimizer='rmsprop',
#              metrics=['accuracy']) #val_acc = 77% train_acc = 83% - 40 epochs - overfitting



In [0]:
def hist(img):
  x = exposure.equalize_hist(img)
  return x

In [0]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rotation_range = 10,
    preprocessing_function = hist)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rotation_range = 10)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

Found 1600 images belonging to 2 classes.
Found 384 images belonging to 2 classes.


In [0]:
model.fit_generator(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=STEP_SIZE_VALID)

model.save_weights('scratch.h5')

Epoch 1/50


/usr/local/lib/python3.6/dist-packages/skimage/exposure/exposure.py:63: UserWarning: This might be a color image. The histogram will be computed on the flattened image. You can instead apply this function to each color channel.
  warn("This might be a color image. The histogram will be "


100/100 [==============================] - 493s 5s/step - loss: 0.6980 - acc: 0.5191 - val_loss: 1.7781 - val_acc: 0.5495
Epoch 2/50
100/100 [==============================] - 478s 5s/step - loss: 0.6930 - acc: 0.5238 - val_loss: 0.7362 - val_acc: 0.5078
Epoch 3/50
100/100 [==============================] - 472s 5s/step - loss: 0.6953 - acc: 0.4722 - val_loss: 0.6965 - val_acc: 0.6224
Epoch 4/50
100/100 [==============================] - 465s 5s/step - loss: 0.6936 - acc: 0.4984 - val_loss: 0.6959 - val_acc: 0.5469
Epoch 5/50
100/100 [==============================] - 470s 5s/step - loss: 0.6947 - acc: 0.4813 - val_loss: 0.6939 - val_acc: 0.5586
Epoch 6/50
100/100 [==============================] - 469s 5s/step - loss: 0.6944 - acc: 0.4950 - val_loss: 0.6946 - val_acc: 0.4714
Epoch 7/50
100/100 [==============================] - 474s 5s/step - loss: 0.6946 - acc: 0.4972 - val_loss: 0.6953 - val_acc: 0.4922
Epoch 8/50
100/100 [==============================] - 466s 5s/step - loss: 0.693

KeyboardInterrupt: ignored